In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna

import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


sns.set_style("darkgrid")

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
loc = pd.read_csv('STORE_LOCATION.csv', sep=';')

In [3]:
display(train.head()), display(test.head()), display(loc.head())

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN


,id,product_rk,store_location_rk,period_start_dt,demand
0,908,40369,317,02.12.2019,NaN
1,909,40370,317,02.12.2019,NaN
2,910,40372,317,02.12.2019,NaN
3,911,40373,317,02.12.2019,NaN
4,912,46272,317,02.12.2019,NaN


,STORE_LOCATION_RK,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK1,STORE_OPEN_DTTM,STORE_CLOSURE_DTTM,STORE_LOCATION_ADK_hashing,STORE_LOCATION_ATTRIB1_hashing,STORE_LOCATION_ATTRIB2_hashing,...,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB14_hashing,STORE_LOCATION_ATTRIB15_hashing,STORE_LOCATION_ATTRIB16_hashing,STORE_LOCATION_ATTRIB17_hashing,STORE_LOCATION_ATTRIB18_hashing,STORE_LOCATION_ATTRIB19_hashing,STORE_LOCATION_ATTRIB20_hashing,STORE_LOCATION_ATTRIB21_hashing
0,277,124,124,19,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,...,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
1,278,124,124,19,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,...,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,0A7D98B3F3BA0A5BC89F693FB146FE40,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
2,279,183,183,20,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,...,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
3,280,211,211,12,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,...,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
4,281,221,221,3,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,...,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91


(None, None, None)

In [4]:
display(train.dtypes, train.shape, train.isna().sum())

Unnamed: 0             int64
product_rk             int64
store_location_rk      int64
period_start_dt       object
demand               float64
PROMO1_FLAG          float64
PROMO2_FLAG          float64
PRICE_REGULAR        float64
PRICE_AFTER_DISC     float64
NUM_CONSULTANT       float64
AUTORIZATION_FLAG    float64
dtype: object

(35344, 11)

Unnamed: 0              0
product_rk              0
store_location_rk       0
period_start_dt         0
demand               1200
PROMO1_FLAG           185
PROMO2_FLAG           185
PRICE_REGULAR        1127
PRICE_AFTER_DISC     1132
NUM_CONSULTANT        185
AUTORIZATION_FLAG     185
dtype: int64

In [5]:
for i in train.columns:
    display(train[i].value_counts())

2047     1
27975    1
15693    1
13644    1
3403     1
        ..
25305    1
31450    1
29403    1
19164    1
0        1
Name: Unnamed: 0, Length: 35344, dtype: int64

40373    6074
40369    6074
40370    6074
40372    6073
46272    6068
96212    4981
Name: product_rk, dtype: int64

798     924
1034    924
1281    924
657     924
764     924
540     924
1202    924
517     924
1005    924
425     924
533     924
525     923
555     923
1162    923
862     923
874     923
453     923
644     923
355     923
866     923
1191    923
1185    923
1080    923
562     923
637     923
317     923
1173    923
380     923
1143    922
1203    922
535     922
1079    922
557     921
504     919
1326    859
1328    726
1316    719
1363    669
1347    648
1380    325
309      15
Name: store_location_rk, dtype: int64

2019-12-02    240
2019-12-09    240
2019-10-21    240
2019-08-26    240
2019-03-11    240
             ... 
2017-01-16    170
2017-02-20    170
2017-01-30    170
2017-02-13    170
2017-01-09    170
Name: period_start_dt, Length: 159, dtype: int64

0.000000     5454
3.000000     2278
2.000000     2268
4.000000     2217
5.000000     1981
             ... 
5.688525        1
36.546448       1
57.453552       1
13.377049       1
22.133333       1
Name: demand, Length: 1806, dtype: int64

...

0.0    35159
Name: PROMO2_FLAG, dtype: int64

1000.000000    5881
3000.000000    5879
500.000000     5879
2000.000000    5879
99.000000      1859
               ... 
164.714286        1
310.285714        1
110.428571        1
74.428571         1
67.000000         1
Name: PRICE_REGULAR, Length: 229, dtype: int64

1000.000000    5881
3000.000000    5879
500.000000     5879
2000.000000    5879
49.000000      1592
               ... 
68.354286         1
48.666667         1
191.857143        1
67.314490         1
192.000000        1
Name: PRICE_AFTER_DISC, Length: 1036, dtype: int64

0.0    35159
Name: NUM_CONSULTANT, dtype: int64

1.0    31913
0.0     3246
Name: AUTORIZATION_FLAG, dtype: int64

In [6]:
# выбрасываю ненужные данные, меняю тип данных
train.drop(['NUM_CONSULTANT', 'PROMO2_FLAG'], axis=1, inplace=True)
train['period_start_dt'] = pd.to_datetime(train['period_start_dt'], format='%Y-%m-%d')
train.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [7]:
loc.drop(loc.columns[2:], axis=1, inplace=True)
loc = loc.rename(str.lower, axis=1)

In [8]:
train = train.merge(loc, how='left', on='store_location_rk')

# Feature engineering 

Для того, чтобы решить задачу с предсказанием временных рядов при помощи мащинного обучения, нужно создать много разных лагированных фичей

In [9]:
# создаю доп. фичи, чтобы уловить сезонные особенности
def create_date_features(train):
    train['month'] = train.period_start_dt.dt.month
#     train['day_of_month'] = train.period_start_dt.dt.day
    train['week_of_month'] = train.period_start_dt.dt.day // 7
#     train['day_of_year'] = train.period_start_dt.dt.dayofyear
    train['week_of_year'] = train.period_start_dt.dt.weekofyear
    train['year'] = train.period_start_dt.dt.year
    return train

In [10]:
create_date_features(train)

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,store_location_lvl_rk4,month,week_of_month,week_of_year,year
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,203,12,2,51,2016
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,203,12,2,51,2016
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,203,12,2,51,2016
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,203,12,2,51,2016
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,203,12,2,51,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0,219,12,4,1,2019
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0,219,12,4,1,2019
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0,219,12,4,1,2019
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0,219,12,4,1,2019


In [11]:
train = train.sort_values(by=['period_start_dt', 'product_rk', 'store_location_rk'])

In [12]:
# лагированные наблюдения
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['demand_lag_' + str(lag)] = dataframe.groupby(["product_rk", "store_location_rk"])['demand'].transform(
            lambda x: x.shift(lag))
    return dataframe

In [13]:
lags = [4, 13, 26, 52, 108]
lag_features(train, lags)

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,store_location_lvl_rk4,month,week_of_month,week_of_year,year,demand_lag_4,demand_lag_13,demand_lag_26,demand_lag_52,demand_lag_108
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,203,12,2,51,2016,NaN,NaN,NaN,NaN,NaN
15,15,40369,317,2016-12-19,50.0,NaN,NaN,NaN,NaN,189,12,2,51,2016,NaN,NaN,NaN,NaN,NaN
938,944,40369,355,2016-12-19,38.0,NaN,NaN,NaN,NaN,54,12,2,51,2016,NaN,NaN,NaN,NaN,NaN
1861,1873,40369,380,2016-12-19,39.0,NaN,NaN,NaN,NaN,189,12,2,51,2016,NaN,NaN,NaN,NaN,NaN
2784,2802,40369,425,2016-12-19,105.0,NaN,NaN,NaN,NaN,203,12,2,51,2016,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975,33150,96212,1326,2019-12-30,NaN,1.0,99.000000,69.300000,1.0,203,12,4,1,2019,NaN,0.0,0.0,21.266667,4.900000
33701,33881,96212,1328,2019-12-30,NaN,1.0,121.021429,84.714286,1.0,54,12,4,1,2019,NaN,0.0,19.0,59.466667,18.923077
34349,34535,96212,1347,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,189,12,4,1,2019,NaN,0.0,20.0,58.670588,NaN
35018,35210,96212,1363,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,26,12,4,1,2019,NaN,0.0,7.0,32.900000,11.000000


In [14]:
# лагированное скользящее среднее
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['demand_roll_mean_' + str(window)] = dataframe.groupby(["product_rk", "store_location_rk"])['demand'].transform(
            lambda x: x.shift(5).rolling(window=window).mean())
    return dataframe

In [15]:
windows = [4, 13, 26, 52, 108]
roll_mean_features(train, windows)

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,store_location_lvl_rk4,...,demand_lag_4,demand_lag_13,demand_lag_26,demand_lag_52,demand_lag_108,demand_roll_mean_4,demand_roll_mean_13,demand_roll_mean_26,demand_roll_mean_52,demand_roll_mean_108
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,40369,317,2016-12-19,50.0,NaN,NaN,NaN,NaN,189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,944,40369,355,2016-12-19,38.0,NaN,NaN,NaN,NaN,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861,1873,40369,380,2016-12-19,39.0,NaN,NaN,NaN,NaN,189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2784,2802,40369,425,2016-12-19,105.0,NaN,NaN,NaN,NaN,203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975,33150,96212,1326,2019-12-30,NaN,1.0,99.000000,69.300000,1.0,203,...,NaN,0.0,0.0,21.266667,4.900000,0.000,0.000000,0.884615,3.024359,3.372840
33701,33881,96212,1328,2019-12-30,NaN,1.0,121.021429,84.714286,1.0,54,...,NaN,0.0,19.0,59.466667,18.923077,19.525,18.530769,24.150000,25.641667,27.747009
34349,34535,96212,1347,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,189,...,NaN,0.0,20.0,58.670588,NaN,3.000,16.615385,18.538462,26.077165,NaN
35018,35210,96212,1363,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,26,...,NaN,0.0,7.0,32.900000,11.000000,1.250,5.615385,8.307692,13.151923,NaN


In [16]:
# лагированное скользящее стандартное отклонение
def roll_std_features(dataframe, windows):
    for window in windows:
        dataframe['demand_roll_std_' + str(window)] = dataframe.groupby(["product_rk", "store_location_rk"])['demand'].transform(
            lambda x: x.shift(5).rolling(window=window).std())
    return dataframe

In [17]:
windows = [4, 13, 26, 52, 108]
roll_std_features(train, windows)

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,store_location_lvl_rk4,...,demand_roll_mean_4,demand_roll_mean_13,demand_roll_mean_26,demand_roll_mean_52,demand_roll_mean_108,demand_roll_std_4,demand_roll_std_13,demand_roll_std_26,demand_roll_std_52,demand_roll_std_108
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,40369,317,2016-12-19,50.0,NaN,NaN,NaN,NaN,189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,944,40369,355,2016-12-19,38.0,NaN,NaN,NaN,NaN,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861,1873,40369,380,2016-12-19,39.0,NaN,NaN,NaN,NaN,189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2784,2802,40369,425,2016-12-19,105.0,NaN,NaN,NaN,NaN,203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975,33150,96212,1326,2019-12-30,NaN,1.0,99.000000,69.300000,1.0,203,...,0.000,0.000000,0.884615,3.024359,3.372840,9.733398e-08,0.000000,1.451259,4.262560,3.917583
33701,33881,96212,1328,2019-12-30,NaN,1.0,121.021429,84.714286,1.0,54,...,19.525,18.530769,24.150000,25.641667,27.747009,6.166665e-01,9.305827,10.068681,21.824965,20.580405
34349,34535,96212,1347,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,189,...,3.000,16.615385,18.538462,26.077165,NaN,6.000000e+00,19.483063,14.461620,22.242199,NaN
35018,35210,96212,1363,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,26,...,1.250,5.615385,8.307692,13.151923,NaN,1.500000e+00,7.263361,6.619784,12.707414,NaN


In [18]:
# лагированное скользящее экспоненциальное среднее
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha) + "_lag_" + str(lag)] = dataframe.groupby(["product_rk", "store_location_rk"])['demand'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

In [19]:
alphas = [0.9, 0.8, 0.7, 0.5, 0.4]
lags = [4, 13, 26, 52, 108]
ewm_features(train, alphas, lags)

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,store_location_lvl_rk4,...,sales_ewm_alpha_0.5_lag_4,sales_ewm_alpha_0.5_lag_13,sales_ewm_alpha_0.5_lag_26,sales_ewm_alpha_0.5_lag_52,sales_ewm_alpha_0.5_lag_108,sales_ewm_alpha_0.4_lag_4,sales_ewm_alpha_0.4_lag_13,sales_ewm_alpha_0.4_lag_26,sales_ewm_alpha_0.4_lag_52,sales_ewm_alpha_0.4_lag_108
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,40369,317,2016-12-19,50.0,NaN,NaN,NaN,NaN,189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,944,40369,355,2016-12-19,38.0,NaN,NaN,NaN,NaN,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1861,1873,40369,380,2016-12-19,39.0,NaN,NaN,NaN,NaN,189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2784,2802,40369,425,2016-12-19,105.0,NaN,NaN,NaN,NaN,203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975,33150,96212,1326,2019-12-30,NaN,1.0,99.000000,69.300000,1.0,203,...,0.000252,0.064542,0.730053,10.810018,7.305512,0.002773,0.165118,1.008419,8.832905,7.717980
33701,33881,96212,1328,2019-12-30,NaN,1.0,121.021429,84.714286,1.0,54,...,19.745785,14.054289,24.738254,69.526814,22.412348,19.587649,16.480274,25.887072,66.681642,22.411728
34349,34535,96212,1347,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,189,...,1.939247,22.447299,16.270752,69.010783,NaN,2.957552,25.229022,16.198154,68.006972,NaN
35018,35210,96212,1363,2019-12-30,NaN,1.0,141.430000,99.000000,1.0,26,...,0.985987,8.412715,9.960041,37.603911,11.000000,1.233186,9.665820,10.106544,36.894276,11.000000


In [20]:
train = pd.get_dummies(train, columns=['PROMO1_FLAG', 'AUTORIZATION_FLAG'])

In [21]:
# использую mean encoding для категориальных переменных
train['product_rk'] = train['product_rk'].map(train.groupby('product_rk')['demand'].mean())
train['store_location_rk'] = train['store_location_rk'].map(train.groupby('store_location_rk')['demand'].mean())
train['store_location_lvl_rk4'] = train['store_location_lvl_rk4'].map(train.groupby('store_location_lvl_rk4')['demand'].mean())
train

,id,product_rk,store_location_rk,period_start_dt,demand,PRICE_REGULAR,PRICE_AFTER_DISC,store_location_lvl_rk4,month,week_of_month,...,sales_ewm_alpha_0.4_lag_4,sales_ewm_alpha_0.4_lag_13,sales_ewm_alpha_0.4_lag_26,sales_ewm_alpha_0.4_lag_52,sales_ewm_alpha_0.4_lag_108,PROMO1_FLAG_0.0,PROMO1_FLAG_1.0,PROMO1_FLAG_2.0,AUTORIZATION_FLAG_0.0,AUTORIZATION_FLAG_1.0
0,0,17.451267,55.666667,2016-12-19,29.0,NaN,NaN,14.454746,12,2,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
15,15,17.451267,9.794074,2016-12-19,50.0,NaN,NaN,12.016786,12,2,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
938,944,17.451267,13.337798,2016-12-19,38.0,NaN,NaN,16.256396,12,2,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1861,1873,17.451267,13.883093,2016-12-19,39.0,NaN,NaN,12.016786,12,2,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
2784,2802,17.451267,18.954918,2016-12-19,105.0,NaN,NaN,14.454746,12,2,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975,33150,8.431571,15.236060,2019-12-30,NaN,99.000000,69.300000,14.454746,12,4,...,0.002773,0.165118,1.008419,8.832905,7.717980,0,1,0,0,1
33701,33881,8.431571,23.014288,2019-12-30,NaN,121.021429,84.714286,16.256396,12,4,...,19.587649,16.480274,25.887072,66.681642,22.411728,0,1,0,0,1
34349,34535,8.431571,24.320586,2019-12-30,NaN,141.430000,99.000000,12.016786,12,4,...,2.957552,25.229022,16.198154,68.006972,NaN,0,1,0,0,1
35018,35210,8.431571,19.225742,2019-12-30,NaN,141.430000,99.000000,8.164348,12,4,...,1.233186,9.665820,10.106544,36.894276,11.000000,0,1,0,0,1


In [22]:
# логарифмирую таргет, чтобы предсказания были положительными (спрос всегда больше 0)
train['demand'] = np.log1p(train["demand"].values)

In [23]:
cols = [col for col in train.columns if col not in ['id', 'demand', 'period_start_dt']]

train1 = train[train['period_start_dt'] < '2019-11-01'].copy()
test1 = train[(train['period_start_dt'] >= '2019-11-01') & (train['period_start_dt'] < '2019-12-01')].copy()

X_train = train1[cols]
y_train = train1['demand']
    
X_val = test1[cols]
y_val = test1['demand']

In [24]:
# подбираю гиперпараметры 
def objective(trial):
    
    param_grid = {
        'boosting_type': 'gbdt',
        'metric': 'mae',
        'n_jobs': -1,
        'n_estimators': 5000,
        'seed': 1,
        'early_stopping_rounds': 200,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3), 
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        
    }
    
    lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=cols)
    lgbval = lgb.Dataset(data=X_val, label=y_val, reference=lgbtrain, feature_name=cols)
    
    model = lgb.train(param_grid, lgbtrain,
                  valid_sets = lgbval,
                  verbose_eval=100)
    
    y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
    loss = mean_absolute_error(np.expm1(y_val), np.expm1(y_pred_val))
#     loss = mean_absolute_error(y_val, y_pred_val)

    return loss

In [ ]:
%%time
study = optuna.create_study(direction="minimize", study_name="LGBM")
study.optimize(objective, n_trials=100)
# я решил не показывать вывод данной строчки кода, т.к. она очень длинная 

In [26]:
study.best_params

{'learning_rate': 0.1338962411650117,
 'lambda_l1': 0.48074511048310864,
 'lambda_l2': 0.0011040836215668386,
 'num_leaves': 9,
 'feature_fraction': 0.7102293153172105,
 'bagging_fraction': 0.9176552911274246,
 'bagging_freq': 4,
 'min_child_samples': 37}

In [27]:
train1 = train[train['period_start_dt'] < '2019-12-01'].copy()
test1 = train[(train['period_start_dt'] >= '2019-12-01') & (train['period_start_dt'] < '2019-12-31')].copy()

X_train = train1[cols]
y_train = train1['demand']
    
X_val = test1[cols]
y_val = test1['demand']

In [28]:
%%time

lgb_params = {'boosting_type': 'gbdt',
              'metric': 'mae',
              'n_jobs': -1,
              'n_estimators': 5000,
              'seed': 1,
#               'early_stopping_rounds': 200,
              'learning_rate': 0.23292247530870516,
              'lambda_l1': 1.1693557095071368e-06,
              'lambda_l2': 0.0056848325061162,
              'num_leaves': 13,
              'feature_fraction': 0.9424769602960624,
              'bagging_fraction': 0.5936670517046974,
              'bagging_freq': 6,
              'min_child_samples': 16
              }

loss = []

    
lgbtrain = lgb.Dataset(data=X_train, label=y_train, feature_name=cols)
#     lgbval = lgb.Dataset(data=X_val, label=y_val, reference=lgbtrain, feature_name=cols)
    
model = lgb.train(lgb_params, lgbtrain,
#                   valid_sets = lgbval,
                  verbose_eval=100)
    
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

c:\users\ed911\appdata\local\programs\python\python38\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10745
[LightGBM] [Info] Number of data points in the train set: 34144, number of used features: 54
[LightGBM] [Info] Start training from score 1.834218
Wall time: 26 s


In [29]:
test1['demand'] = np.exp(y_pred_val)
test1.sort_values('id', inplace=True)
test1['demand'].describe()

count    1200.000000
mean       17.457624
std        23.045698
min         0.498230
25%         3.445080
50%         8.328894
75%        22.724739
max       179.118344
Name: demand, dtype: float64

In [30]:
submission_df = test1.loc[:, ['id', 'demand']]
submission_df['predicted'] = submission_df['demand'].copy()
submission_df.drop('demand', axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)

In [31]:
submission_df

,id,predicted
908,908,6.442410
909,909,9.960334
910,910,5.360724
911,911,4.951014
912,912,3.687183
...,...,...
35339,35537,33.335397
35340,35538,34.411558
35341,35539,24.536677
35342,35540,3.521956
